<h1 style="font-size:30px;">Motion Detection in Videos</h1> 

In this notebook, we are going to demonstrate how to detect motion in videos that contain moving foreground objects. Topics covered in this notebook include:

* Using a background subtraction model to isolate moving objects in the foreground (creating a foreground mask)
* Using a method called "erosion" to remove noise from the foreground mask.

Background subtraction models work by building a statistical model of the background scene based on previous frames in a video stream. The model can then be used to create a foreground mask of the current frame by comparing the current frame with the model of the background. If the current frame is of the "same" background scene then the foreground mask should theoretically be entirely black. If anything changes in the scene then the foreground mask will detect the changes and record them as non-zero pixel intensities where the intensity in the foreground mask is a measure of the change in the scene. The foreground mask can therefore be used to identify motion in a video stream. In this notebook we will identify the motion by drawing a bounding rectangle that encompasses all the non-zero pixels in the foreground mask.

Erosion is a morphological operation that has many uses in computer vision. One use of erosion is to remove small amounts of noise in binary images.

# 1. Preview Video

In [1]:
import cv2
import numpy as np


# 2. Work Flow

* Create a statistical model of the background scene using **`createBackgroundSubtractorKNN()`**
* For each video frame:
    * Compare the current frame with the background model to create a foreground mask using the **`apply()`** method
    * Apply erosion to the foreground mask to reduce noise using **`erode()`**
    * Find the bounding rectangle that encompasses all the non-zero points in the foreground mask using  **`boundingRect()`**


## 2.1 Workflow Preview

### <font style="color:rgb(50,120,230)">Static scene (no motion in the video)</font>
The top left frame (Foreground Mask) is almost entirely black with the exception of a few pixels that have non-zero intensity. Even though nothing apparent is changing in the scene from frame to frame, adjacent frames in a video stream may have very subtle differences due to the way the camera sensor is perceiving the scene, or the scene itself could be changing ever so slightly from lighting variations (reflections, shadows, etc.). The red bounding rectangle shown to the right encompasses all the non-zero pixels in the foreground mask. Note that some non-zero pixels are not even visible since a single pixel is extremely small in this high-resolution image.

The bottom left frame (Foreground Mask Eroded) shows the effect of applying erosion to the Foreground Mask (eliminating the noise entirely).

![Still-2-secc](https://opencv.org/wp-content/uploads/2021/08/c0-m4-Still-2sec-03.jpg)

The examples below illustrate how erosion works. We begin by defining a small structuring element called a kernel. In this example, we are using a 3x3 kernel. The center of the kernel (yellow dot) is placed over every pixel ***p*** in the original image. At each pixel location, ***p***, we then examine the surrounding pixels beneath the kernel. If ALL the pixels beneath the kernel are white then the pixel ***p*** in the eroded image remains white.  Otherwise, the pixel in the eroded image is set to black. 

![Erosion](https://opencv.org/wp-content/uploads/2021/08/c0-m4-Erosion-03.jpg)

### <font style="color:rgb(50,120,230)">Dynamic scene (obvious motion in the video)</font>
The top left frame (Foreground Mask) is detecting large changes in the scene due to the motion of the book, but the foreground mask also contains a lot of background noise from subtle lighting changes (e.g., shadows).

The bottom left frame (Foreground Mask Eroded) shows the effect of applying erosion to the Foreground Mask. Most of the noise associated with shadows has been suppressed.

![Feature-Image-03a](https://opencv.org/wp-content/uploads/2021/08/c0-m4-Feature-Image-03a.jpg)
![Feature-Image-03b](https://opencv.org/wp-content/uploads/2021/08/c0-m4-Feature-Image-03b.jpg)

# 3. Documentation Summary

## 3.1 KNN Background Subtractor

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

**`createBackgroundSubtractorKNN()`** creates KNN Background Subtractor.

### <font color="green">Function Syntax </font>
``` python
retval = cv2.createBackgroundSubtractorKNN([, history[, dist2Threshold[, detectShadows]]])	
```

`retval`: KNN Background Subtractor object.

The function has 3 optional arguments:

1. `history` Length of the history.
2. `dist2Threshold` Threshold on the squared distance between the pixel and the sample to decide whether a pixel is close to that sample. This parameter does not affect the background update.
3. `detectShadows` If true, the algorithm will detect shadows and mark them. It decreases the speed a bit, so if you do not need this feature, set the parameter to false.


### <font color="green">OpenCV Documentation</font>

[**`createBackgroundSubtractorKNN()`**](https://docs.opencv.org/4.5.2/de/de1/group__video__motion.html#gac9be925771f805b6fdb614ec2292006d)

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

## 3.2 Applying Background Subtractor

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

**`apply()`** computes a foreground mask.

### <font color="green">Function Syntax </font>
``` python
fgmask = cv2.BackgroundSubtractor.apply(image[, fgmask[, learningRate]])	
```

The function has **1 required input argument** and 2 optional flags:

1. `image` Next video frame.
2. `fgmask` The output foreground mask as an 8-bit binary image.
3. `learningRate` The value between 0 and 1 that indicates how fast the background model has learned. Negative parameter value makes the algorithm to use some automatically chosen learning rate. 0 means that the background model is not updated at all, 1 means that the background model is completely reinitialized from the last frame.

### <font color="green">OpenCV Documentation</font>

[**`apply()`**](https://docs.opencv.org/4.5.2/d7/df6/classcv_1_1BackgroundSubtractor.html#aa735e76f7069b3fa9c3f32395f9ccd21)<br>

<hr style="border:none; height: 4px; background-color:#D3D3D3" />


## 3.3 Erosion

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

**`erode()`** erodes an image by using a specific structuring element.

### <font style = "color:rgb(8,133,37)">Function Syntax </font>
``` python
dst = cv2.erode(src, kernel[, dst[, anchor[, iterations[, borderType[, borderValue]]]]])	
```

`dst`: output image of the same size and type as src. 

The function has **2 required input arguments**:

1. `src` input image; the number of channels can be arbitrary, but the depth should be one of CV_8U, CV_16U, CV_16S, CV_32F or CV_64F.
2. `kernel`	structuring element used for erosion; if element = Mat(), a 3 x 3 rectangular structuring element is used. Kernel can be created using getStructuringElement.


### <font color="green">OpenCV Documentation</font>

[**`erode()`**](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb)<br>

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

## 3.4 findNonZero

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

**`findNonZero()`** Returns the list of locations of non-zero pixels.


### <font color="green">Function Syntax </font>
``` python
retval = cv2.findNonZero(src)	
```

The function has **1 required input argument**:

1. `src` single-channel array.

### <font color="green">OpenCV Documentation</font>

[**`findNonZero()`**](https://docs.opencv.org/4.5.2/d2/de8/group__core__array.html#gaed7df59a3539b4cc0fe5c9c8d7586190) <br>

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

## 3.5 Bounding Rectangle

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

**`boundingRect()`** calculates and returns the minimal up-right bounding rectangle for the specified point set or non-zero pixels of gray-scale image.

### <font color="green">Function Syntax </font>
``` python
retval = cv2.boundingRect(array)	

retval is a tuple that contains four values: (x, y, w, h)
    
    (x,y) are the coordinates for the upper left coner of the bounding rectatngle
    (w,h) are the width and height of the bounding rectangle
    
The function can also be called to explicitly return the individual values:
    
x, y, w, h  = cv2.boundingRect(array)  
        
```

The function has **1 required input argument**:

1. `array` Input gray-scale image or 2D point set, stored in std::vector or [Mat](https://docs.opencv.org/4.5.2/d3/d63/classcv_1_1Mat.html).

### <font color="green">OpenCV Documentation</font>

[**`boundingRect()`**](https://docs.opencv.org/4.5.2/d3/dc0/group__imgproc__shape.html#ga103fcbda2f540f3ef1c042d6a9b35ac7) <br>

<hr style="border:none; height: 4px; background-color:#D3D3D3" />